In [1]:
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import numpy_indexed as npi

In [2]:
input_reader = JsonReader("logs2")

2022-12-16 11:07:50,862	WARNING json_reader.py:238 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs2/*.json', '/tf/Cage/Notebooks/logs2/*.zip']


Find target sequence in dataset

In [3]:
num_actions = 145
episode_length = 100
actions = np.zeros(num_actions)
input_reader = JsonReader("logs2")
target = np.zeros((1,52+num_actions))
#target[0,52+43] = 1
target = np.load('target.npy')
new_state = {}

for i in range(1000):
    data = input_reader.next()
    vec = np.zeros((episode_length, num_actions))
    vec[np.arange(episode_length),data['actions']] = 1
    state_action_data = np.concatenate([data['obs'], vec], axis=1)
    mask = np.flatnonzero(npi.contains(target, state_action_data))
    for m in mask:
        if m == episode_length-1: continue
        if not data['obs'][m+1].tobytes() in new_state:
            new_state[data['obs'][m+1].tobytes()] = 1
        else:
            new_state[data['obs'][m+1].tobytes()] += 1

2022-12-16 11:07:50,875	WARNING json_reader.py:238 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs2/*.json', '/tf/Cage/Notebooks/logs2/*.zip']


View next state from sequence

In [4]:
for s in new_state.keys():
    print(new_state[s])
    print(np.frombuffer(s))

print(len(new_state.keys()))

1
[0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
7
[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
46
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
26
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
6
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
8
[0. 0. 0. 0. 0. 0. 0. 0.

In [5]:
new_state_c = {}

prediction = np.load('prediction.npy')
for i in range(100):
    state = np.zeros(52)
    for i in range(52):
        state[i] = np.array(np.random.rand() < prediction[i], dtype=np.int8)
    if not state.tobytes() in new_state_c:
        new_state_c[state.tobytes()] = 1
    else:
        new_state_c[state.tobytes()] += 1
            
for s in new_state_c.keys():
    print(new_state_c[s])
    print(np.frombuffer(s))

print(len(new_state_c.keys()))

1
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
34
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
6
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0.]
1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
2
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
1
[0. 0. 0. 0. 0. 0. 0. 0. 

In [6]:
2**52

4503599627370496

In [7]:
new_state = {}

for i in range(10000):
    data = input_reader.next()
    for i in range(100):
        if not data['obs'][i].tobytes() in new_state:
            new_state[data['obs'][i].tobytes()] = 1
        else:
            new_state[data['obs'][i].tobytes()] += 1
    
print(len(new_state.keys()))

6560


In [8]:
6617*155

1025635